<a href="https://colab.research.google.com/github/mohamedbakhet/Medicine-In-Take-Reminder-and-Monitor/blob/master/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tensorflow
! pip install h5py

In [ ]:
from keras.models import model_from_json
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import numpy as np
import pandas as pd
from random import shuffle
import warnings
warnings.filterwarnings('ignore')
from keras.layers import Input
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import Sequential,models
from keras.models import Model
from tensorflow.keras import layers
from keras.layers import Input
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten,MaxPooling2D,Conv1D,MaxPooling1D,add,Activation,GlobalAveragePooling2D,BatchNormalization,ReLU,MaxPool2D,SeparableConv2D,Add,GlobalAvgPool2D
#from keras.utils import plot_model
from keras.layers.merge import concatenate
from keras.utils.np_utils import to_categorical
import os,keras
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


In [ ]:
#load train data set
training_data=[]
for index,value in enumerate (os.listdir('/content/drive/MyDrive/train_data')): #path to the data set on google drive
  print(value)
  lis=np.load('/content/drive/MyDrive/train_data/'+value)
  for j in lis[:1600]:
    training_data.append([np.array(j),value[:-4]])
shuffle(training_data)

#load test data set
test_data=[]
for index,value in enumerate (os.listdir('/content/drive/MyDrive/train_data')):
  print(value)
  lis=np.load('/content/drive/MyDrive/train_data/'+value)
  for j in lis[:400]:
    test_data.append([np.array(j),value[:-4]])
shuffle(test_data)

In [ ]:
#split data into train_test & encoding target

X_train = np.array([i[0] for i in training_data]).reshape(-1,300,15,1)
Y_train = np.array([i[1] for i in training_data])
train_labels_set = np.unique(Y_train,axis=0)
train_labels = LabelEncoder()
train_labels = train_labels.fit(Y_train)
Y_train = to_categorical(train_labels.transform(Y_train))

X_test = np.array([i[0] for i in test_data]).reshape(-1,300,15,1)
Y_test = np.array([i[1] for i in test_data])
test_labels_set = np.unique(Y_test,axis=0)
test_labels = LabelEncoder()
test_labels = test_labels.fit(Y_test)
Y_test = to_categorical(test_labels.transform(Y_test))

In [ ]:
#ResNet50
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X
def ResNet50(input_shape=(224, 224, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model
base_model = ResNet50(input_shape=(300, 15, 1))
headModel = base_model.output
headModel = Flatten()(headModel)
headModel=Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense( 14,activation='sigmoid', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)
model = Model(inputs=base_model.input, outputs=headModel)



In [ ]:
#get the gpu name 
device_name = tf.test.gpu_device_name() #you can ignore this if don't have to use the gpu during the training proccess

In [ ]:
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
with tf.device(device_name): #in case that you skipped the gpu device comment these two lines and uncomment the last one and run
  hist = model.fit(X_train, Y_train, epochs=10,validation_data=(X_test,Y_test))
#hist = model.fit(X_train, Y_train, epochs=10,validation_data=(X_test,Y_test))

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("ResNet50.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("ResNet50.h5")

In [ ]:
# load json and create model
json_file = open('ResNet50.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("ResNet50.h5")

In [ ]:
#to evaluate the model
opt = Adam(lr=0.001)
loaded_model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
y_predict = loaded_model.predict(X_test)
print(classification_report(Y_test.argmax(axis=1),y_predict.argmax(axis=1),target_names=test_labels_set))